In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_18551/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104224,167194,https://arxiv.org/abs/2308.10248,[2308.10248] Activation Addition: Steering Lan...,\N,\N,2023-09-02 17:57:28.92159+00,2023-09-02 17:57:28.921603+00,NaN,\N,Custom,29.0,0.0,0
104225,167207,https://www.lesswrong.com/posts/fJqP9WcnHXBRBe...,Meta Questions about Metaphilosophy,\N,\N,2023-09-02 17:57:28.987001+00,2023-09-02 17:57:28.987006+00,NaN,\N,Custom,72.0,0.0,0
104226,167212,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,www.centreforeffectivealtruism.org,2023-09-02 17:08:06.43648+00,2023-09-03 12:40:09.528732+00,0.0,\N,CEA,54.0,0.0,\N
104227,167213,https://www.noahpinion.blog/p/hey-tech-folks-v...,Hey tech folks: Vivek Ramaswamy is not the one,\N,\N,2023-09-03 12:39:55.570223+00,2023-09-03 12:39:55.570223+00,NaN,\N,Noah,69.0,0.0,\N


In [2]:
links['added'].max()

'2023-09-03 12:39:55.571479+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
104221    167183
104222    167184
104223    167185
104224    167194
104225    167207
Name: id, Length: 13155, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
104228,167214,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-09-03 12:39:55.571479+00,2023-09-03 12:39:55.571479+00,NaN,\N,CEA,23.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

104228    167214
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': 0,
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/1373
...11/1373
...21/1373
...31/1373
...41/1373
...51/1373
...61/1373
...71/1373
...81/1373
...91/1373
...101/1373
...111/1373
...121/1373
...131/1373
...141/1373
...151/1373
...161/1373
...171/1373
...181/1373
...191/1373
...201/1373
...211/1373
...221/1373
...231/1373
...241/1373
...251/1373
...261/1373
...271/1373
...281/1373
...291/1373
...301/1373
...311/1373
...321/1373
...331/1373
...341/1373
...351/1373
...361/1373
...371/1373
...381/1373
...391/1373
...401/1373
...411/1373
...421/1373
...431/1373
...441/1373
...451/1373
...461/1373
...471/1373
...481/1373
...491/1373
...501/1373
...511/1373
...521/1373
...531/1373
...541/1373
...551/1373
...561/1373
...571/1373
...581/1373
...591/1373
...601/1373
...611/1373
...621/1373
...631/1373
...641/1373
...651/1373
...661/1373
...671/1373
...681/1373
...691/1373
...701/1373
...711/1373
...721/1373
...731/1373
...741/1373
...751/1373
...761/1373
...771/1373
...781/1373
...791/1373
...801/1373
...811/1373
...821/1373
...831/1373
...

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105596,168582,https://twitter.com/yanda_chen_/status/1681412...,"Yanda Chen on X: ""[1/9] Large Language Models ...",\N,\N,2023-09-03 13:42:10.827308,2023-09-03 13:42:10.827313,\N,\N,Custom,78.0,0.0,0
105597,168583,https://openai.com/blog/how-should-ai-systems-...,How should AI systems behave and who should de...,\N,\N,2023-09-03 13:42:10.832718,2023-09-03 13:42:10.832722,\N,\N,Custom,11.0,0.0,0
105598,168584,https://www.interconnects.ai/,(1) Interconnects,\N,\N,2023-09-03 13:42:10.838238,2023-09-03 13:42:10.838242,\N,\N,Custom,23.0,0.0,0
105599,168585,https://arxiv.org/abs/2308.08708,[2308.08708] Consciousness in Artificial Intel...,\N,\N,2023-09-03 13:42:10.843758,2023-09-03 13:42:10.843763,\N,\N,Custom,66.0,0.0,0


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105596,168582,https://twitter.com/yanda_chen_/status/1681412...,"Yanda Chen on X: ""[1/9] Large Language Models ...",\N,\N,2023-09-03 13:42:10.827308,2023-09-03 13:42:10.827313,\N,\N,Custom,78.0,0.0,0
105597,168583,https://openai.com/blog/how-should-ai-systems-...,How should AI systems behave and who should de...,\N,\N,2023-09-03 13:42:10.832718,2023-09-03 13:42:10.832722,\N,\N,Custom,11.0,0.0,0
105598,168584,https://www.interconnects.ai/,(1) Interconnects,\N,\N,2023-09-03 13:42:10.838238,2023-09-03 13:42:10.838242,\N,\N,Custom,23.0,0.0,0
105599,168585,https://arxiv.org/abs/2308.08708,[2308.08708] Consciousness in Artificial Intel...,\N,\N,2023-09-03 13:42:10.843758,2023-09-03 13:42:10.843763,\N,\N,Custom,66.0,0.0,0


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('\\', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links['starred'] = links['starred'].fillna(0).apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int) # Backwards compatibility
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism, every voter can v...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13, 6:40 PM: I'm driving East out of S...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105596,168582,https://twitter.com/yanda_chen_/status/1681412...,"Yanda Chen on X: ""[1/9] Large Language Models ...",N,\N,2023-09-03 13:42:10.827308,2023-09-03 13:42:10.827313,\N,\N,Custom,78,0,0
105597,168583,https://openai.com/blog/how-should-ai-systems-...,How should AI systems behave and who should de...,N,\N,2023-09-03 13:42:10.832718,2023-09-03 13:42:10.832722,\N,\N,Custom,11,0,0
105598,168584,https://www.interconnects.ai/,(1) Interconnects,N,\N,2023-09-03 13:42:10.838238,2023-09-03 13:42:10.838242,\N,\N,Custom,23,0,0
105599,168585,https://arxiv.org/abs/2308.08708,[2308.08708] Consciousness in Artificial Intel...,N,\N,2023-09-03 13:42:10.843758,2023-09-03 13:42:10.843763,\N,\N,Custom,66,0,0


In [10]:
links.to_csv('data/export.csv', index=False)